# LSTM sin hacer mucho, 0.75635 en Kaggle. 

In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pandas as pd

Using TensorFlow backend.


In [18]:
train = pd.read_csv(r'https://raw.githubusercontent.com/Ignacio-Ibarra/NLP-Disasters/intento_3_nachos/intento_3/train_clean.csv')
test = pd.read_csv(r'https://raw.githubusercontent.com/Ignacio-Ibarra/NLP-Disasters/intento_3_nachos/intento_3/test_clean.csv')

In [21]:
train_text = train['text_clean_nosw']
y_train = train['target']
test_text = test['text_clean_nosw']
id_test = test['id']

In [57]:
complete = train_text.append(test_text)

In [45]:
uniques = []
for i in list(complete.values): 
  for j in str(i).split():
    if j not in uniques:
      uniques.append(j)
numwords = len(uniques)
print('The number of unique words is {}'.format(numwords))



The number of unique words is 17140


In [58]:
complete = [str(x) for x in complete.values]

In [60]:
tokenizer = Tokenizer(num_words=17140)
tokenizer.fit_on_texts(list(complete))

In [62]:
train_text = [str(x) for x in train_text.values]
test_text = [str(x) for x in test_text.values]

In [64]:
train_text= tokenizer.texts_to_sequences(train_text)
test_text= tokenizer.texts_to_sequences(test_text) 

In [65]:
# We'll apply a padding method to add zeros and set the fixed size into each vector.

xtrain=pad_sequences(train_text,padding='post')
xtest=pad_sequences(test_text,padding='post')

In [68]:
# Defining the LSTM model

# We apply the Embedding layer for input data before adding the LSTM layer into the Keras sequential model. The model definition goes as a following.
vocab_size=len(tokenizer.word_index)+1
embedding_dim=50
model=Sequential()
model.add(layers.Embedding(input_dim=vocab_size,
      output_dim=embedding_dim,
      input_length=23))
model.add(layers.LSTM(units=50,return_sequences=True))
model.add(layers.LSTM(units=10))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(8))
model.add(layers.Dense(1, activation="sigmoid"))
model.compile(optimizer="adam", loss="binary_crossentropy", 
     metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 23, 50)            857050    
_________________________________________________________________
lstm_5 (LSTM)                (None, 23, 50)            20200     
_________________________________________________________________
lstm_6 (LSTM)                (None, 10)                2440      
_________________________________________________________________
dropout_3 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 9         
Total params: 879,787
Trainable params: 879,787
Non-trainable params: 0
________________________________________________

In [69]:
#Finally, we'll train the model and check the training accuracy.

model.fit(xtrain,y_train, epochs=20, batch_size=16, verbose=False)

loss, acc = model.evaluate(xtrain, y_train, verbose=False)
print("Training Accuracy: ", round(acc,2))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Training Accuracy:  0.99


In [81]:
#Predicting test data

#Finally, we can predict test data and check the prediction accuracy.

ypred=model.predict(xtest)

ypred[ypred>0.5]=1 
ypred[ypred<=0.5]=0 

In [83]:
ypred = [int(x) for x in ypred]

In [84]:
df = pd.DataFrame()
df['id'] = [x for x in id_test.values]
df['target'] = ypred
df.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [85]:
df.to_csv('16.submit.csv', index=False)